<a href="https://colab.research.google.com/github/caffeinated4ighs/Forest_Fires_satellite/blob/main/Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install rasterio


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 58.2 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import rasterio
from sklearn.model_selection import train_test_split
import os
from tensorflow.keras import models, layers

In [ ]:
dataset_dir = '/content/drive/MyDrive/dataset'

In [ ]:
subdirs = ['1', '2']
#1 is fire and 2 is non-fire
img_paths = []
labels = []

for subdir in subdirs:
    subdir_path = os.path.join(dataset_dir, subdir)
    for img in os.listdir(subdir_path):
        img_paths.append(os.path.join(subdir_path, img))
        labels.append(subdir)

img_paths = np.array(img_paths)
labels = np.array(labels)

# Split the data into 70% training and 15% testing and 15% validation
X_train, X_temp, y_train, y_temp = train_test_split(img_paths, labels, test_size=0.3, random_state = 42)
X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size = 0.5, random_state = 69)


In [ ]:
print(X_train.shape)
print(X_test.shape)
print(X_val.shape)

(54,)
(12,)
(12,)


In [ ]:
def read_geotiffs(file_paths):
    data = []
    for file_path in file_paths:
        with rasterio.open(file_path) as src:
            #bands from the GeoTIFF
            bands = []
            for band in range(1, src.count + 1):  # Loop through each band
                band_data = src.read(band)
                bands.append(band_data)
            # Stack bands together
            image = np.stack(bands, axis=-1)
            data.append(image)
    return np.array(data)

X_train = read_geotiffs(X_train)
X_test = read_geotiffs(X_test)
X_val = read_geotiffs(X_val)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(X_val.shape)

(54, 512, 512, 6)
(12, 512, 512, 6)
(12, 512, 512, 6)


In [ ]:
num_images, height, width, num_bands = X_train.shape
x_train_reshaped = X_train.reshape(-1, num_bands)

min_vals = np.min(x_train_reshaped, axis=0)
max_vals = np.max(x_train_reshaped, axis=0)

max_vals[max_vals == min_vals] = min_vals[max_vals == min_vals] + 1.0

x_train_normalized = (x_train_reshaped - min_vals) / (max_vals - min_vals)

x_train_normalized = x_train_normalized.reshape(num_images, height, width, num_bands)

In [ ]:
print(x_train_normalized.shape)


(54, 512, 512, 6)


In [ ]:
num_images, height, width, num_bands = X_test.shape
x_test_reshaped = X_test.reshape(-1, num_bands)

min_vals = np.min(x_test_reshaped, axis=0)
max_vals = np.max(x_test_reshaped, axis=0)

max_vals[max_vals == min_vals] = min_vals[max_vals == min_vals] + 1.0

x_test_normalized = (x_test_reshaped - min_vals) / (max_vals - min_vals)

x_test_normalized = x_test_normalized.reshape(num_images, height, width, num_bands)

In [ ]:
print(x_test_normalized.shape)

(12, 512, 512, 6)


In [ ]:
num_images, height, width, num_bands = X_val.shape
x_val_reshaped = X_val.reshape(-1, num_bands)

min_vals = np.min(x_val_reshaped, axis=0)
max_vals = np.max(x_val_reshaped, axis=0)


max_vals[max_vals == min_vals] = min_vals[max_vals == min_vals] + 1.0

x_val_normalized = (x_val_reshaped - min_vals) / (max_vals - min_vals)

x_val_normalized = x_val_normalized.reshape(num_images, height, width, num_bands)

In [ ]:
print(x_val_normalized.shape)

(12, 512, 512, 6)


In [ ]:

print(x_train_normalized.shape)
print(x_test_normalized.shape)
print(x_val_normalized.shape)

X_train = x_train_normalized
X_test = x_test_normalized
X_val = x_val_normalized

(54, 512, 512, 6)
(12, 512, 512, 6)
(12, 512, 512, 6)


In [ ]:
#labels to binary class
y_train = y_train.astype('float32')
y_test = y_test.astype('float32')
y_val = y_val.astype('float32')


In [ ]:
#CNN architecture
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(512, 512, 6)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())

model.add(layers.Dense(64, activation='relu'))

model.add(layers.Dense(1, activation='sigmoid'))


model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

# Train
history1 = model.fit(X_train, y_train, epochs=10, batch_size=5, validation_data=(X_test, y_test))

# Evaluate
test_loss, test_acc = model.evaluate(X_val, y_val)
print(f'Test accuracy: {test_acc}')

model.save('/content/drive/MyDrive/Networks_WnB/CNN')

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 510, 510, 32)      1760      
                                                                 
 max_pooling2d (MaxPooling2  (None, 255, 255, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 253, 253, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 126, 126, 64)      0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 124, 124, 64)      36928     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 62, 62, 64)        0

In [ ]:
#U_NET
inputs = layers.Input(shape=(512,512,6))

# Encoder
conv1 = layers.Conv2D(64, 3, activation='relu', padding='same')(inputs)
pool1 = layers.MaxPooling2D(pool_size=(2, 2))(conv1)

conv2 = layers.Conv2D(128, 3, activation='relu', padding='same')(pool1)
pool2 = layers.MaxPooling2D(pool_size=(2, 2))(conv2)

conv3 = layers.Conv2D(256, 3, activation='relu', padding='same')(pool2)
pool3 = layers.MaxPooling2D(pool_size=(2, 2))(conv3)

conv4 = layers.Conv2D(521, 3, activation='relu', padding='same')(pool3)
pool4 = layers.MaxPooling2D(pool_size=(2, 2))(conv4)

# Decoder
up5 = layers.Conv2D(512, 2, activation='relu', padding='same')(layers.UpSampling2D(size=(2, 2))(pool4))
merge5 = layers.Concatenate(axis=3)([conv4, up5])

up6 = layers.Conv2D(256, 2, activation='relu', padding='same')(layers.UpSampling2D(size=(2, 2))(merge5))
merge6 = layers.Concatenate(axis=3)([conv3, up6])

up7 = layers.Conv2D(128, 2, activation='relu', padding='same')(layers.UpSampling2D(size=(2, 2))(merge6))
merge7 = layers.Concatenate(axis=3)([conv2, up7])

up8 = layers.Conv2D(64, 2, activation='relu', padding='same')(layers.UpSampling2D(size=(2, 2))(merge7))
merge8 = layers.Concatenate(axis=3)([conv1, up8])

# Output layer
output = layers.Conv2D(1, 1, activation='sigmoid')(merge8)

models.Model(inputs=inputs, outputs=output)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

history2 = model.fit(X_train, y_train, epochs=10, batch_size=5, validation_data=(X_val, y_val))

test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_acc}')

model.save('/content/drive/MyDrive/Networks_WnB/UNET')

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 510, 510, 32)      1760      
                                                                 
 max_pooling2d (MaxPooling2  (None, 255, 255, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 253, 253, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 126, 126, 64)      0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 124, 124, 64)      36928     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 62, 62, 64)        0